In [223]:
import pandas as pd
import numpy as np
from statistics import mean
import typing
from typing import List
from typing import Dict
import copy
import math
import random


In [14]:
snakeobject1 = {
  "id": "sn1",
  "name": "sn1",
  "health": 100,
  "body": [{"x": 1, "y": 0}, {"x": 1, "y": 1}, {"x": 1, "y": 2}],
  "latency": "123",
  "head": {"x": 1, "y": 0},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [138]:
snakeobject1c = {
  "id": "sn2",
  "name": "sn2",
  "health": 100,
  "body": [{"x": 1, "y": 0}, {"x": 2, "y":0}, {"x": 3, "y": 0}],
  "latency": "123",
  "head": {"x": 1, "y": 0},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

snakeobject1o = {
  "id": "sn2",
  "name": "sn2",
  "health": 100,
  "body": [{"x": -1, "y": 0}, {"x": 0, "y":0}, {"x": 0, "y": 1}],
  "latency": "123",
  "head": {"x": -1, "y": 0},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [15]:
snakeobject2 = {
  "id": "sn2",
  "name": "sn2",
  "health": 100,
  "body": [{"x": 2, "y": 1}, {"x": 3, "y": 1}, {"x": 4, "y": 1}],
  "latency": "123",
  "head": {"x": 2, "y": 1},
  "length": 3,
  "shout": "s!",
  "squad": "2",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [16]:
snakeobject3 = {
  "id": "sn3",
  "name": "sn1",
  "health": 100,
  "body": [{"x": 5, "y": 5}, {"x": 5, "y": 4}, {"x": 5, "y": 3}],
  "latency": "123",
  "head": {"x": 5, "y": 5},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [17]:
snakeobject4 = {
  "id": "sn4",
  "name": "sn2",
  "health": 100,
  "body": [{"x": 7, "y": 6}, {"x": 7, "y": 7}, {"x": 7, "y": 8}],
  "latency": "123",
  "head": {"x": 7, "y": 6},
  "length": 3,
  "shout": "s!",
  "squad": "2",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [18]:
board1 = {"height": 11, "width": 11,
  "food": [{"x": 2, "y": 0}],
  "hazards": [{"x": 2, "y": 2}],
  "snakes": [snakeobject1, snakeobject2]}

In [19]:
board2 = {"height": 11, "width": 11,
  "food": [{}],
  "hazards": [{}],
  "snakes": [snakeobject3, snakeobject4]}

In [22]:
board3 = {"height": 11, "width": 11,
  "food": [{"x": 2, "y": 0}],
  "hazards": [{"x": 2, "y": 2}],
  "snakes": [snakeobject1, snakeobject2, snakeobject3, snakeobject4]}

In [398]:
class Snake:
    def __init__(self,snake_json: Dict):       
        self.id = snake_json['id']
        self.name = snake_json['name']
        self.health = snake_json['health']
        self.body = snake_json['body']
        self.latency = snake_json['latency']
        self.head = snake_json['head']
        self.length = snake_json['length']
        self.shout = snake_json['shout']
        self.squad = snake_json['squad']
        self.customizations = snake_json['customizations']

    def move(self, foods: List[Dict], move: str) -> 'Snake':
        x = copy.deepcopy(self)
        new_body = []
        if move == 'up':
            new_body = [{'x': self.head['x'], 'y': self.head['y']+1}] + self.body
        elif move == 'down':
            new_body = [{'x': self.head['x'], 'y': self.head['y']-1}] + self.body
        elif move == 'right':
            new_body = [{'x': self.head['x']+1, 'y': self.head['y']}] + self.body
        elif move == 'left':
            new_body = [{'x': self.head['x']-1, 'y': self.head['y']}] + self.body

        if new_body[0] in foods:
            x.body = new_body
            x.head = new_body[0]
        else:
            x.body = new_body[:-1]
            x.head = new_body[0]

        return x

    def get_snake(self) -> Dict:
        json =  {
            'id':self.id,
            'name':self.name,
            'health':self.health,
            'body':self.body,
            'latency': self.latency,
            'head': self.head,
            'length': self.length,
            'shout': self.shout,
            'squad': self.squad,
            'customizations': self.customizations
            }
        return json

class Board:
    def __init__(self,board_json: Dict):
        self.height = board_json['height']
        self.width = board_json['width']
        self.food = board_json['food']
        self.hazards = board_json['hazards']
        self.snakes = board_json['snakes']

    def permute_ours(self, my_snake: Snake, safe_moves: List) -> List['Board']:
        new_boards = []
        my_index = next((index for (index, d) in enumerate(self.snakes) if d["id"] == my_snake.id), None)

        for m in safe_moves:
            nboard = copy.deepcopy(self)
            new_snake = my_snake.move(foods = self.food, move = m)
            nboard.snakes[my_index] = new_snake.get_snake()
            nboard.food = [f for f in nboard.food if f != new_snake.head]
            new_boards += [nboard]

        return new_boards

    def permute_theirs(self, my_snake: Snake) -> List['Board']:
        new_snakes = [[Snake(s) for s in self.snakes if s['id'] == my_snake.id]]
        new_boards = []

        # Create new snakes
        for sn in self.snakes:
            if sn['id'] == my_snake.id:
                pass
            else: 
                new_snakes_tmp = []
                for m in ['up','down','left','right']:
                    new_snake = [Snake(sn).move(foods = self.food, move = m)]
                    msnakes = [s+new_snake for s in new_snakes]
                    new_snakes_tmp += msnakes
                new_snakes = new_snakes_tmp
        
        # create new boards using the new snakes
        for los in new_snakes:
            new_boards += [self.filter_snakes_to_board(los)]

        return new_boards

    def lolox_to_lox(lolox: List[List]) -> List:
        lox = []
        for l in lolox:
            lox += l
        return lox

    def filter_snakes_to_board(self, snakes: List["Snake"]) -> List["Board"]:
        
        # filter snakes if its head is out of bounds / in hazard / in itself
        # self.height / self.width / self.hazards / s.body[1:]
        filtered_snakes = [sn for sn in snakes if ((sn.head not in self.hazards) and 
                                                    (sn.head['x'] >= 0) and
                                                    (sn.head['x'] <= (self.width - 1)) and
                                                    (sn.head['y'] >= 0) and
                                                    (sn.head['y'] <= (self.height - 1)) and
                                                    (sn.head not in sn.body[1:]))]

        # remove food if eaten
        # self.food
        snake_heads = [sn.head for sn in snakes]
        filtered_food = [f for f in self.food if f not in snake_heads]
       
        # remove snakes that have collided with others
        non_collided_snakes = [sn.get_snake() for sn in filtered_snakes if sn.head not in lolox_to_lox([s.body for s in filtered_snakes if s != sn])]
        
        # create new board with the snakes and foods
        nboard = copy.deepcopy(self)
        nboard.food = filtered_food

        nboard.snakes = non_collided_snakes
        return nboard
    

    def score(self, my_snake: Snake) -> float:
        death_aversion = 0
        hunger_aversion = 0
        aggression = 0      
        
        local_ids = [s['id'] for s in self.snakes]
        other_snakes = [s for s in self.snakes if s['id'] != my_snake.id]

        if my_snake.get_snake()['id'] not in local_ids:
            death_aversion = -10000
            factors = [death_aversion, hunger_aversion, aggression]
            return mean(factors)
        
        else:
            local_my_snake = [s for s in self.snakes if s['id'] == my_snake.id][0]
            hunger_aversion = (local_my_snake['health'] - 100)*100    
            aggression = len(other_snakes)*-100
            factors = [death_aversion, hunger_aversion, aggression]
            return mean(factors)

    def score_n_steps(self, my_snake: Snake, moves: List, counter: float, n: float) -> float:
        counter += 1
        #initiate dictionary

        scores = []

        for move in moves:
            # Create all boards based on our move + their moves
            our_move_boards = self.permute_ours(my_snake,[move])
            all_possible_next_boards = []
            for b in our_move_boards:
                all_possible_next_boards += b.permute_theirs(my_snake)

            # Creating unique list out of boards
            unique_boards = [] 
            unique_snakes = []

            for b in all_possible_next_boards:
                if b.snakes not in unique_snakes:
                    unique_boards += [b]
                    unique_snakes += [b.snakes]

            ## Score test
            for b in unique_boards:
                local_score = b.score(my_snake)
                scores += [local_score]
                
                if counter < n:
                    try:
                        scores += [b.score_n_steps(my_snake, ['up','down','right','left'], counter, n)]
                    except:
                        pass

        return mean(scores)

    def score_moves(self, my_snake: Snake, moves: List, counter: float, n: float) -> Dict:
        scored_moves = {}
        for move in moves:
            scored_moves[move] = self.score_n_steps(my_snake, [move], counter, n)

        return scored_moves

    def build_np(self) -> np.ndarray:
        # Use the enum values to populate a 2d array
        pass

class BoardTile():
    pass

class BoardTree:
    def __init__(init_state: Board):
    # Build the tree from our initial state by permuting
        pass        

In [399]:
s1 = Snake(snakeobject1)
s1c = Snake(snakeobject1c) # collide with s1
s1o = Snake(snakeobject1o) # goes out of bounds
s2 = Snake(snakeobject2)
s3 = Snake(snakeobject3)
s4 = Snake(snakeobject4)
b1 = Board(board1)
b2 = Board(board2)
b3 = Board(board3)

In [413]:
## score_moves TEST
test_snake = s1
test_board = b3
summary = test_board.score_moves(test_snake, ['up','down','left','right'],0,2)
summary

{'up': -3333.3333333333335,
 'down': -3333.3333333333335,
 'left': -1063.7880940082646,
 'right': -1689.346590909091}

In [439]:
x = pd.DataFrame(list(summary.items()),columns=['moves','scores'])

In [395]:
## score_n_steps TEST
test_snake = s1
test_board = b3
test_board.score_n_steps(test_snake, ['up'],0,3)

-3333.3333333333335

In [364]:
## my snake move score 1 step ahead TEST
my_snake_0 = test_snake
pob = test_board.permute_ours(my_snake_0,['left'])

ptlob = []
for b in pob:
    ptlob += b.permute_theirs(my_snake_0)

counter = 1
for b in ptlob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]


## Creating unique list out of combo-ed list of boards
unique_ptlob = [] 
unique_snakes = []

for b in ptlob:
    if b.snakes not in unique_snakes:
        unique_ptlob += [b]
        unique_snakes += [b.snakes]

## Score test
counter = 1
score = []
for b in unique_ptlob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]
    local_score = b.score(my_snake_0)
    score += [local_score]
    print('{} | score: {} | id: {} | heads: {}'.format(counter, local_score, ids, heads))
    counter += 1

mean(score)

1 | score: -33.333333333333336 | id: ['sn1', 'sn3'] | heads: [{'x': 0, 'y': 0}, {'x': 5, 'y': 6}]
2 | score: -66.66666666666667 | id: ['sn1', 'sn2', 'sn3'] | heads: [{'x': 0, 'y': 0}, {'x': 2, 'y': 0}, {'x': 5, 'y': 6}]
3 | score: 0 | id: ['sn1'] | heads: [{'x': 0, 'y': 0}]
4 | score: -33.333333333333336 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 0}, {'x': 2, 'y': 0}]
5 | score: -33.333333333333336 | id: ['sn1', 'sn3'] | heads: [{'x': 0, 'y': 0}, {'x': 4, 'y': 5}]
6 | score: -66.66666666666667 | id: ['sn1', 'sn2', 'sn3'] | heads: [{'x': 0, 'y': 0}, {'x': 2, 'y': 0}, {'x': 4, 'y': 5}]
7 | score: -33.333333333333336 | id: ['sn1', 'sn3'] | heads: [{'x': 0, 'y': 0}, {'x': 6, 'y': 5}]
8 | score: -66.66666666666667 | id: ['sn1', 'sn2', 'sn3'] | heads: [{'x': 0, 'y': 0}, {'x': 2, 'y': 0}, {'x': 6, 'y': 5}]
9 | score: -66.66666666666667 | id: ['sn1', 'sn3', 'sn4'] | heads: [{'x': 0, 'y': 0}, {'x': 5, 'y': 6}, {'x': 7, 'y': 5}]
10 | score: -100 | id: ['sn1', 'sn2', 'sn3', 'sn4'] | heads: [{'x

-66.66666666666667

In [315]:
## my snake move score 1 step ahead TEST
my_snake_0 = s1
pob = b1.permute_ours(my_snake_0,['left'])

ptlob = []
for b in pob:
    ptlob += b.permute_theirs(my_snake_0)

counter = 1
for b in ptlob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]
    #print('{} | id: {} | heads: {}'.format(counter,ids,heads))
    #counter += 1

## Creating unique list out of combo-ed list of boards
unique_ptlob = [] 
unique_snakes = []

for b in ptlob:
    if b.snakes not in unique_snakes:
        unique_ptlob += [b]
        unique_snakes += [b.snakes]

## Score test
counter = 1
score = []
for b in unique_ptlob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]
    local_score = b.score(my_snake_0)
    score += [local_score]
    print('{} | score: {} | id: {} | heads: {}'.format(counter, local_score, ids, heads))
    counter += 1

mean(score)

1 | score: 0 | id: ['sn1'] | heads: [{'x': 0, 'y': 0}]
2 | score: -33.333333333333336 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 0}, {'x': 2, 'y': 0}]


-16.666666666666668

In [253]:
## combo test
pob = b1.permute_ours(s1,['up','down','left'])
ptlob = []
counter = 1
for b in pob:
    ptlob += b.permute_theirs(s3)
    
for b in ptlob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]
    print('{} | id: {} | heads: {}'.format(counter,ids,heads))
    counter += 1

1 | id: ['sn1'] | heads: [{'x': 1, 'y': 2}]
2 | id: [] | heads: []
3 | id: ['sn1'] | heads: [{'x': 0, 'y': 1}]
4 | id: ['sn1'] | heads: [{'x': 2, 'y': 1}]
5 | id: ['sn1', 'sn2'] | heads: [{'x': 1, 'y': 2}, {'x': 2, 'y': 0}]
6 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
7 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 1}, {'x': 2, 'y': 0}]
8 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
9 | id: ['sn1'] | heads: [{'x': 1, 'y': 2}]
10 | id: ['sn2'] | heads: [{'x': 1, 'y': 1}]
11 | id: ['sn1'] | heads: [{'x': 0, 'y': 1}]
12 | id: [] | heads: []
13 | id: ['sn1'] | heads: [{'x': 1, 'y': 2}]
14 | id: [] | heads: []
15 | id: ['sn1'] | heads: [{'x': 0, 'y': 1}]
16 | id: ['sn1'] | heads: [{'x': 2, 'y': 1}]
17 | id: [] | heads: []
18 | id: [] | heads: []
19 | id: [] | heads: []
20 | id: [] | heads: []
21 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
22 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
23 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
24 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
25 | id: ['sn2'] | heads: 

In [254]:
## Creating unique list out of combo-ed list of boards
unique_ptlob = [] 
unique_snakes = []
counter = 1

for b in ptlob:
    if b.snakes not in unique_snakes:
        unique_ptlob += [b]
        unique_snakes += [b.snakes]
    
for b in unique_ptlob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]
    print('{} | id: {} | heads: {}'.format(counter,ids,heads))
    counter += 1

1 | id: ['sn1'] | heads: [{'x': 1, 'y': 2}]
2 | id: [] | heads: []
3 | id: ['sn1'] | heads: [{'x': 0, 'y': 1}]
4 | id: ['sn1'] | heads: [{'x': 2, 'y': 1}]
5 | id: ['sn1', 'sn2'] | heads: [{'x': 1, 'y': 2}, {'x': 2, 'y': 0}]
6 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
7 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 1}, {'x': 2, 'y': 0}]
8 | id: ['sn2'] | heads: [{'x': 1, 'y': 1}]
9 | id: ['sn1'] | heads: [{'x': 0, 'y': 1}]
10 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 1}, {'x': 2, 'y': 0}]
11 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 1}, {'x': 1, 'y': 1}]


In [257]:
## Score test
my_snake_0 = s1
counter = 1
for b in unique_ptlob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]
    score = b.score(my_snake_0)
    print('{} | score: {} | id: {} | heads: {}'.format(counter, score, ids, heads))
    counter += 1

1 | score: 0 | id: ['sn1'] | heads: [{'x': 1, 'y': 2}]
2 | score: -3333.3333333333335 | id: [] | heads: []
3 | score: 0 | id: ['sn1'] | heads: [{'x': 0, 'y': 1}]
4 | score: 0 | id: ['sn1'] | heads: [{'x': 2, 'y': 1}]
5 | score: -33.333333333333336 | id: ['sn1', 'sn2'] | heads: [{'x': 1, 'y': 2}, {'x': 2, 'y': 0}]
6 | score: -3333.3333333333335 | id: ['sn2'] | heads: [{'x': 2, 'y': 0}]
7 | score: -33.333333333333336 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 1}, {'x': 2, 'y': 0}]
8 | score: -3333.3333333333335 | id: ['sn2'] | heads: [{'x': 1, 'y': 1}]
9 | score: 0 | id: ['sn1'] | heads: [{'x': 0, 'y': 1}]
10 | score: -33.333333333333336 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 1}, {'x': 2, 'y': 0}]
11 | score: -33.333333333333336 | id: ['sn1', 'sn2'] | heads: [{'x': 0, 'y': 1}, {'x': 1, 'y': 1}]


In [222]:
## filter_snakes_to_board test
nb = b3.filter_snakes_to_board([s1,s1c, s1o, s2])
for s in nb.snakes:
    print(s['id'])

sn2


In [183]:
## permute_theirs test
b3 = Board(board3)
lob = b3.permute_theirs(s2)
counter = 1
for b in lob:
    ids = [s['id'] for s in b.snakes]
    heads = [s['head'] for s in b.snakes]
    print('{} | id: {} | heads {} '.format(counter, ids,heads))
    counter += 1

1 | id: ['sn2', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 5, 'y': 6}] 
2 | id: ['sn2', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 5, 'y': 6}] 
3 | id: ['sn2', 'sn1', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 0, 'y': 0}, {'x': 5, 'y': 6}] 
4 | id: ['sn2', 'sn1', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 2, 'y': 0}, {'x': 5, 'y': 6}] 
5 | id: ['sn2'] | heads [{'x': 2, 'y': 1}] 
6 | id: ['sn2'] | heads [{'x': 2, 'y': 1}] 
7 | id: ['sn2', 'sn1'] | heads [{'x': 2, 'y': 1}, {'x': 0, 'y': 0}] 
8 | id: ['sn2', 'sn1'] | heads [{'x': 2, 'y': 1}, {'x': 2, 'y': 0}] 
9 | id: ['sn2', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 4, 'y': 5}] 
10 | id: ['sn2', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 4, 'y': 5}] 
11 | id: ['sn2', 'sn1', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 0, 'y': 0}, {'x': 4, 'y': 5}] 
12 | id: ['sn2', 'sn1', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 2, 'y': 0}, {'x': 4, 'y': 5}] 
13 | id: ['sn2', 'sn3'] | heads [{'x': 2, 'y': 1}, {'x': 6, 'y': 5}] 
14 | id: ['sn2', 'sn3'] | heads [{'x': 2, 'y': 1}

In [188]:
## permnute_ours test
lob = b1.permute_ours(s2,['down','right','left']) #s2's down will consume food
counter = 1

print('origin')

for s in b1.snakes:
    print(' - id: {} | head: {} | body: {}'.format(s['id'],s['head'],s['body']))

for b in lob:
    print('board #{}'.format(counter))
    print(' - food: {}'.format(b.food))
    for s in b.snakes:
        print(' - id: {} | head: {} | body: {}'.format(s['id'],s['head'],s['body'])) #check movement
    
    counter += 1

for b in lob:
    print(b.food) #checks food

origin
 - id: sn1 | head: {'x': 1, 'y': 0} | body: [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}]
 - id: sn2 | head: {'x': 2, 'y': 1} | body: [{'x': 2, 'y': 1}, {'x': 3, 'y': 1}, {'x': 4, 'y': 1}]
board #1
 - food: []
 - id: sn1 | head: {'x': 1, 'y': 0} | body: [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}]
 - id: sn2 | head: {'x': 2, 'y': 0} | body: [{'x': 2, 'y': 0}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}, {'x': 4, 'y': 1}]
board #2
 - food: [{'x': 2, 'y': 0}]
 - id: sn1 | head: {'x': 1, 'y': 0} | body: [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}]
 - id: sn2 | head: {'x': 3, 'y': 1} | body: [{'x': 3, 'y': 1}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}]
board #3
 - food: [{'x': 2, 'y': 0}]
 - id: sn1 | head: {'x': 1, 'y': 0} | body: [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}]
 - id: sn2 | head: {'x': 1, 'y': 1} | body: [{'x': 1, 'y': 1}, {'x': 2, 'y': 1}, {'x': 3, 'y': 1}]
[]
[{'x': 2, 'y': 0}]
[{'x': 2, 'y': 0}]


In [63]:
my_snake = s1
[s for s in b1.snakes if s['id'] == my_snake.id]

[{'id': 'sn1',
  'name': 'sn1',
  'health': 100,
  'body': [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}],
  'latency': '123',
  'head': {'x': 1, 'y': 0},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}}]

In [76]:
# testing combo of permute ours + permute theirs
# currently should be outputting lolos. TODO lolob.
lob = b3.permute_ours(s2,['down','right','left'])
bt = []
counter = 1

for b in lob:
    bt += b.permute_theirs(s2)

print(s2.head)

for los in bt:
    #print(s for s in los)
    tmp_id = []
    tmp_head = []
    for s in los:
       tmp_id += [s.id]
       tmp_head += [s.head]
       
    print('{}: {} | {}'.format(counter,tmp_id,tmp_head))
    counter += 1

{'x': 2, 'y': 1}
1: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 1, 'y': 1}, {'x': 5, 'y': 6}, {'x': 7, 'y': 7}]
2: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 1, 'y': -1}, {'x': 5, 'y': 6}, {'x': 7, 'y': 7}]
3: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 0, 'y': 0}, {'x': 5, 'y': 6}, {'x': 7, 'y': 7}]
4: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 2, 'y': 0}, {'x': 5, 'y': 6}, {'x': 7, 'y': 7}]
5: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 1, 'y': 1}, {'x': 5, 'y': 4}, {'x': 7, 'y': 7}]
6: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 1, 'y': -1}, {'x': 5, 'y': 4}, {'x': 7, 'y': 7}]
7: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 0, 'y': 0}, {'x': 5, 'y': 4}, {'x': 7, 'y': 7}]
8: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 2, 'y': 0}, {'x': 5, 'y': 4}, {'x': 7, 'y': 7}]
9: ['sn2', 'sn1', 'sn3', 'sn4'] | [{'x': 2, 'y': 0}, {'x': 1, 'y': 1}, {'x': 4, 'y': 5}, {'x': 7, 'y': 7}]
10: ['sn2', 'sn1',

In [41]:
body = [{"x": 0, "y": 0}, 
        {"x": 1, "y": 0}, 
        {"x": 2, "y": 0}]

In [2]:
body2 = [{"x": 0, "y": 0}, 
        {"x": 1, "y": 0}, 
        {"x": 2, "y": 0}]

In [3]:
body3 = [{"x": 0, "y": 0}, 
        {"x": 1, "y": 0}, 
        {"x": 2, "y": 3}]

In [4]:
opponents = [body,body2,body3]
op_body = []

In [5]:
for x in opponents:
    op_body += x[:-1]

In [6]:
body + [{'x': 9, 'y':9}]

[{'x': 0, 'y': 0}, {'x': 1, 'y': 0}, {'x': 2, 'y': 0}, {'x': 9, 'y': 9}]

In [ ]:
'''
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
'''

In [ ]:
'''
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L L L L L
L 1 X L L L L L L L L
L 1 < 2 2 L L L L L L
L V F L L L L L L L L
'''

In [273]:
snakeobject11 = {
  "id": "sn1",
  "name": "sn1",
  "health": 100,
  "body": [{"x": 1, "y": 0}, {"x": 1, "y": 1}, {"x": 1, "y": 2}],
  "latency": "123",
  "head": {"x": 1, "y": 0},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [274]:
snakeobject21 = {
  "id": "sn2",
  "name": "sn2",
  "health": 100,
  "body": [{"x": 2, "y": 1}, {"x": 3, "y": 1}, {"x": 4, "y": 1}],
  "latency": "123",
  "head": {"x": 2, "y": 1},
  "length": 3,
  "shout": "s!",
  "squad": "2",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [275]:
snakeobject12 = {
  "id": "sn3",
  "name": "sn3",
  "health": 100,
  "body": [{"x": 5, "y": 5}, {"x": 5, "y": 4}, {"x": 5, "y": 3}],
  "latency": "123",
  "head": {"x": 5, "y": 5},
  "length": 3,
  "shout": "s!",
  "squad": "1",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [276]:
snakeobject22 = {
  "id": "sn4",
  "name": "sn4",
  "health": 100,
  "body": [{"x": 7, "y": 6}, {"x": 7, "y": 7}, {"x": 7, "y": 8}],
  "latency": "123",
  "head": {"x": 7, "y": 6},
  "length": 3,
  "shout": "s!",
  "squad": "2",
  "customizations":{"color":"#26CF04","head":"smile","tail":"bolt"}}

In [277]:
board1 = {"height": 11, "width": 11,
  "food": [{"x": 2, "y": 0}],
  "hazards": [{"x": 2, "y": 2}],
  "snakes": [snakeobject11, snakeobject21]}

In [278]:
board2 = {"height": 11, "width": 11,
  "food": [{}],
  "hazards": [{}],
  "snakes": [snakeobject12, snakeobject22]}

In [279]:
board3 = {"height": 11, "width": 11,
  "food": [{"x": 2, "y": 0}],
  "hazards": [{"x": 2, "y": 2}],
  "snakes": [snakeobject11, snakeobject21, snakeobject12, snakeobject22]}

In [ ]:
'''
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L 4 L L L
L L L L L L L X L L L
L L L L L L L X L L L
L L L L L ^ L L L L L
L L L L L 3 L L L L L
L L L L L 3 L L L L L
L 1 X L L L L L L L L
L 1 < 2 2 L L L L L L
L V F L L L L L L L L
'''

['up', 'left', 'down', 'right']

In [ ]:
snake_name = "sn3"
safe_moves = ['up','left','down','right']

# Testing next_lob

In [350]:
def next_lob(board, lolos, counter, branch):
    # board: board object
    # lolos: running list of snakes  #listof (listof snakes)
    # counter: counter for each recursive loop
    # branch: indicates my snake's moves. one of: 'left' 'right' 'up' 'down'
    
    # PULL ITEMS OUT OF THE BOARD OBJECT
    snakes = board["snakes"]             # listof [sn, sn, ...] - list of snakes on the board
    snake = snakes[counter:counter+1][0] # [{"id": , ... }.{"id": , ...}] - snake which is going to be making the turn
    hazards = board["hazards"]           # listof [({'x':_,'y':_}),...] - list of coordinates of hazards
    board_height = board["height"]       # Natural
    board_width = board["width"]         # Natural
    foods = board["food"]                # listof [({'x':_,'y':_}),...] - list of coordinates of foods
    
    # INITIATE MOVE
    # lolop_all4 is listof (listof {x,y})
    # lolop_all4 represents all of the next possible moves for a given snake
    # if the snake is my snake, it will simply generate the specific safe move as represented by branch
    lolop_all4 = []

    if (snake['name'] ==  snake_name):
        if branch == 'up':
            lolop_all4.append([{"x": snake['head']['x'] , "y": snake['head']['y']+1}] + snake['body'][:])
        elif branch == 'down':
            lolop_all4.append([{"x": snake['head']['x'] , "y": snake['head']['y']-1}] + snake['body'][:])
        elif branch == 'left':
            lolop_all4.append([{"x": snake['head']['x']-1, "y": snake['head']['y']}] + snake['body'][:])
        elif branch == 'right':
            lolop_all4.append([{"x": snake['head']['x']+1, "y": snake['head']['y']}] + snake['body'][:])

    else: lolop_all4 = [
            [{"x": snake['head']['x'] , "y": snake['head']['y']+1}] + snake['body'][:],
            [{"x": snake['head']['x'] , "y": snake['head']['y']-1}] + snake['body'][:],
            [{"x": snake['head']['x']-1 , "y": snake['head']['y']}] + snake['body'][:],
            [{"x": snake['head']['x']+1 , "y": snake['head']['y']}] + snake['body'][:]
        ]
    
    # HAZARD / OUT OF BOUNDS / IMPLODE
    # lolop is a listof (listof {x,y})
    # lolop screens out any snakes that have hit a hazard or is out of bounds
    lolop = []
    for p in lolop_all4:
        if (not ((p[0] in hazards) or
            (p[0]['x'] < 0) or
            (p[0]['x'] > (board_width - 1)) or 
            (p[0]['y'] < 0) or
            (p[0]['y'] > (board_height - 1)) or
            (p[0] in p[1:]))):

            lolop.append(p)

    # FOOD
    # lolop_food_adj is a listof (listof {x,y})
    # foods_not_eaten is a listof {x,y}
    # lolop_food_adj includes snakes adjusted for whether it ate a food or not
    # foods_not_eaten is foods, with eaten foods removed 
    lolop_food_adj = []
    foods_not_eaten = foods
    for p in lolop:
        if p[0] in foods:
            lolop_food_adj.append(p)
        else:
            lolop_food_adj.append(p[:-1])
        
    for f in foods:
        for p in lolop:
            if f in p:
                foods_not_eaten.remove(f)

    # CONVERT POSITION TO SNAKE DATA
    # lolop_to_lolos is a listof (listof snakes)
    # lolop_to_lolos takes lolop_food_adj and produce list of list of snakes
    lolop_to_lolos = []
    for pos in lolop_food_adj:
        lolop_to_lolos.append(
            [{
                "id": snake['id'],
                "name": snake['name'],
                "health": 54,
                "body": pos,
                "latency": snake['latency'],
                "head": pos[0],
                "length": len(pos),
                "shout": snake['shout'],
                "squad": snake['squad'],
                "customizations": snake['customizations']
            }]
        )

    # APPEND NEW SNAKE TO OLD LISTOF SNAKES
    # next_lolos is a listof (listof snake)
    # append new snakes to old list of snakes
    next_lolos = [] #[[sn, sn, ...], [sn, sn, ...], [sn, sn, ...]]

    if len(lolos) == 0:
        next_lolos = lolop_to_lolos
    else:    
        for los in lolos:                           #starting list
            losb = []
            for s in los:                           #each snake already processed
                losb.append(s['body'])              #add to listof snake bodies
                
            for s in lolop_to_lolos:                #new snake
                if (s[0]['head'] not in losb[0]):   #append if not collided
                    next_lolos.append(los + s)


    # TEST OF RECURSION
    # basecase condition: if we have looped through every snake on the board
    # convert the list of snakes to board representations
    # and return the board
    counter += 1
    if counter == len(snakes):
        lob = []
        for los in next_lolos:
            lob.append(
                {"height": board_height,
                "width": board_width,
                "food": foods_not_eaten,
                "hazards": hazards,
                "snakes": los 
                }
            )
        return lob

    # Trust the natural recursion <3
    return next_lob(board, next_lolos, counter, branch)

In [351]:
board = board3
new_lob = next_lob(board, [], 0, 'left')
len(new_lob)

3

In [ ]:
new_safe_moves = safe_moves
new_new_lob = []
for b in next_lob(board, [], 0, 'left'):
    my_snake = [snake for snake in b['snakes'] if snake['name'] == snake_name]
    
    if len(my_snake) > 0:
        for m in new_safe_moves[b]: #new_safe_moves fn required
            new_new_lob += next_lob(b, [], 0 , m)

len(new_new_lob)

In [367]:
score = mean([score_b(b) for b in new_lolob])
score

-1458.3333333333335

# Testing score_b

In [344]:
from statistics import mean

def score_b(board):
    my_snake = [snake for snake in board['snakes'] if snake['name'] == snake_name]

    #my_state: am i alive
    my_state = 0
    if len(my_snake) == 0:
        my_state = -10000 
    
    #distance to food: the closer the better
    distance = 0
    
    #n_enemies: the lower the better
    n_enemies = 0

    factors = [my_state, distance, n_enemies]
    score = mean(factors)
    return score

In [345]:
score_lob(lob)
scores = {}

for move in safe_moves:
    score = mean([score_b(b) for b in next_lob(board3, [], 0 , move)])
    scores[move] = score

scores

{'up': 0, 'left': 0, 'down': -3333.3333333333335, 'right': 0}

In [466]:
snake_name = 'sn1'

In [467]:
moves = []
scores = []
summary = pd.DataFrame()

for move in safe_moves:
    new_new_lob = []
    for b in next_lob(board, [], 0, move):
        my_snake = [snake for snake in b['snakes'] if snake['name'] == snake_name]
        
        if len(my_snake) > 0:
            for move2 in safe_moves: #TODO: new_safe_moves fn required
                new_new_lob += next_lob(b, [], 0 , move2)

    score = [score_b(b) for b in new_new_lob]
    print('length of scores for {} branch is {}'.format(move, len(score)))

    if len(score) > 0:
        moves.append(move)
        scores.append(mean(score))
    
summary["moves"] = moves
summary["scores"] = scores
summary

length of scores for up branch is 0
length of scores for left branch is 567
length of scores for down branch is 0
length of scores for right branch is 0


,moves,scores
0,left,-2857.142857


In [468]:
max_score = max(summary['scores'])
best_next_moves = []
for i in range(len(summary)):
    if summary['scores'][i] == max_score:
        best_next_moves.append(summary['moves'][i])


best_next_moves

['left']

In [450]:
summary.iloc[2]

moves       right
scores   -833.333
Name: 2, dtype: object

In [ ]:
next_best_moves = summary

In [ ]:
''' board3
L L L L L L L L L L L
L L L L L L L L L L L
L L L L L L L 4 L L L
L L L L L L L 4 L L L
L L L L L L L V L L L
L L L L L A L L L L L
L L L L L 3 L L L L L
L L L L L 3 L L L L L
L 1 X L L L L L L L L
L 1 < 2 2 L L L L L L
L V F L L L L L L L L
'''

['up', 'left', 'down', 'right']

In [469]:
def score_b(board):
    my_snake = [snake for snake in board['snakes'] if snake['name'] == snake_name]

    #death_aversion: am i alive
    death_aversion = 0
    hunger_aversion = 0
    agression = 0

    if len(my_snake) == 0:
      death_aversion = -1000
    elif len(my_snake) == 1:
      hunger_aversion = my_snake[0]['length']
      
      agression = -100 * len(board['snakes'].remove(my_snake[0]))
    
    #distance to food: the closer the better
    distance = 0
    

    factors = [death_aversion, distance, agression, hunger_aversion]
    score = mean(factors)
    return score

In [487]:
snake_name

'sn1'

In [499]:
board3 = {"height": 11, "width": 11,
  "food": [{"x": 2, "y": 0}],
  "hazards": [{"x": 2, "y": 2}],
  "snakes": [snakeobject11, snakeobject21, snakeobject12, snakeobject22]}

In [500]:
board = board3

## Make new safe moves

In [516]:
my_snake

[{'id': 'sn1',
  'name': 'sn1',
  'health': 100,
  'body': [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}],
  'latency': '123',
  'head': {'x': 1, 'y': 0},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}}]

In [509]:
board

{'height': 11,
 'width': 11,
 'food': [{'x': 2, 'y': 0}],
 'hazards': [{'x': 2, 'y': 2}],
 'snakes': [{'id': 'sn1',
   'name': 'sn1',
   'health': 100,
   'body': [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}],
   'latency': '123',
   'head': {'x': 1, 'y': 0},
   'length': 3,
   'shout': 's!',
   'squad': '1',
   'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
  {'id': 'sn2',
   'name': 'sn2',
   'health': 100,
   'body': [{'x': 2, 'y': 1}, {'x': 3, 'y': 1}, {'x': 4, 'y': 1}],
   'latency': '123',
   'head': {'x': 2, 'y': 1},
   'length': 3,
   'shout': 's!',
   'squad': '2',
   'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
  {'id': 'sn3',
   'name': 'sn3',
   'health': 100,
   'body': [{'x': 5, 'y': 5}, {'x': 5, 'y': 4}, {'x': 5, 'y': 3}],
   'latency': '123',
   'head': {'x': 5, 'y': 5},
   'length': 3,
   'shout': 's!',
   'squad': '1',
   'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
  {'id': 

[{'x': 1, 'y': 0},
 {'x': 1, 'y': 1},
 {'x': 1, 'y': 2},
 {'x': 2, 'y': 1},
 {'x': 3, 'y': 1},
 {'x': 4, 'y': 1},
 {'x': 5, 'y': 5},
 {'x': 5, 'y': 4},
 {'x': 5, 'y': 3},
 {'x': 7, 'y': 6},
 {'x': 7, 'y': 7},
 {'x': 7, 'y': 8}]

In [537]:
board_height = 11
board_width = 11

In [553]:
loh = board['hazards']
lob = sum([sn['body'][:-1] for sn in board['snakes']],[])

low = []

max_height = board_height
max_width = board_width
for h in range(board_height+1):
    low.append({"x":-1, "y":h})
    low.append({"x":max_width, "y":h})

for w in range(board_width+1):
    low.append({"x":w, "y":-1})
    low.append({"x":w, "y":max_height})

lod = (lob + lob + low)

In [563]:
next_pos = my_snake[0]["head"]

next_moves = [{'move': 'left', 'pos': {"x": next_pos["x"]-1, "y": next_pos["y"]}},
    {'move': 'right', 'pos': {"x": next_pos["x"]+1, "y": next_pos["y"]}},
    {'move': 'down', 'pos': {"x": next_pos["x"], "y": next_pos["y"]-1}},
    {'move': 'up', 'pos': {"x": next_pos["x"], "y": next_pos["y"]+1}}]

In [564]:
safe_next_moves = [m['move'] for m in next_moves if m['pos'] not in lod]
unsafe_next_moves = [m['move'] for m in next_moves if m['pos'] in lod]

In [518]:
if len(safe_next_moves) == 0:
    safe_next_moves = unsafe_next_moves



{'x': 1, 'y': 0}

In [501]:
board['snakes']

[{'id': 'sn1',
  'name': 'sn1',
  'health': 100,
  'body': [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}],
  'latency': '123',
  'head': {'x': 1, 'y': 0},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn2',
  'name': 'sn2',
  'health': 100,
  'body': [{'x': 2, 'y': 1}, {'x': 3, 'y': 1}, {'x': 4, 'y': 1}],
  'latency': '123',
  'head': {'x': 2, 'y': 1},
  'length': 3,
  'shout': 's!',
  'squad': '2',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn3',
  'name': 'sn3',
  'health': 100,
  'body': [{'x': 5, 'y': 5}, {'x': 5, 'y': 4}, {'x': 5, 'y': 3}],
  'latency': '123',
  'head': {'x': 5, 'y': 5},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn4',
  'name': 'sn4',
  'health': 100,
  'body': [{'x': 7, 'y': 6}, {'x': 7, 'y': 7}, {'x': 7, 'y': 8}],
  'latency': '123',
  'h

In [502]:
my_snake = [snake for snake in board['snakes'] if snake['name'] == snake_name]
my_snake

[{'id': 'sn1',
  'name': 'sn1',
  'health': 100,
  'body': [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}],
  'latency': '123',
  'head': {'x': 1, 'y': 0},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}}]

In [506]:
tmp = board['snakes'].copy()
tmp

[{'id': 'sn1',
  'name': 'sn1',
  'health': 100,
  'body': [{'x': 1, 'y': 0}, {'x': 1, 'y': 1}, {'x': 1, 'y': 2}],
  'latency': '123',
  'head': {'x': 1, 'y': 0},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn2',
  'name': 'sn2',
  'health': 100,
  'body': [{'x': 2, 'y': 1}, {'x': 3, 'y': 1}, {'x': 4, 'y': 1}],
  'latency': '123',
  'head': {'x': 2, 'y': 1},
  'length': 3,
  'shout': 's!',
  'squad': '2',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn3',
  'name': 'sn3',
  'health': 100,
  'body': [{'x': 5, 'y': 5}, {'x': 5, 'y': 4}, {'x': 5, 'y': 3}],
  'latency': '123',
  'head': {'x': 5, 'y': 5},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn4',
  'name': 'sn4',
  'health': 100,
  'body': [{'x': 7, 'y': 6}, {'x': 7, 'y': 7}, {'x': 7, 'y': 8}],
  'latency': '123',
  'h

In [507]:
tmp.remove(my_snake[0])

In [508]:
tmp

[{'id': 'sn2',
  'name': 'sn2',
  'health': 100,
  'body': [{'x': 2, 'y': 1}, {'x': 3, 'y': 1}, {'x': 4, 'y': 1}],
  'latency': '123',
  'head': {'x': 2, 'y': 1},
  'length': 3,
  'shout': 's!',
  'squad': '2',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn3',
  'name': 'sn3',
  'health': 100,
  'body': [{'x': 5, 'y': 5}, {'x': 5, 'y': 4}, {'x': 5, 'y': 3}],
  'latency': '123',
  'head': {'x': 5, 'y': 5},
  'length': 3,
  'shout': 's!',
  'squad': '1',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}},
 {'id': 'sn4',
  'name': 'sn4',
  'health': 100,
  'body': [{'x': 7, 'y': 6}, {'x': 7, 'y': 7}, {'x': 7, 'y': 8}],
  'latency': '123',
  'head': {'x': 7, 'y': 6},
  'length': 3,
  'shout': 's!',
  'squad': '2',
  'customizations': {'color': '#26CF04', 'head': 'smile', 'tail': 'bolt'}}]